In [507]:
import os 
import sys
import datetime
import csv 
from functools import partial

import cirq
import qsimcirq
import openfermion as of
import numpy as np
from scipy.sparse import linalg
import multiprocessing as mp
Pi=3.1415


$$ \begin{align} H = &- t \sum_{\langle i,j \rangle} \sum_{\sigma} (a^\dagger_{i, \sigma} a_{j, \sigma} + a^\dagger_{j, \sigma} a_{i, \sigma})


     + U \sum_{i} a^\dagger_{i, \uparrow} a_{i, \uparrow}
                 a^\dagger_{i, \downarrow} a_{i, \downarrow}
    \\
    &- \mu \sum_i \sum_{\sigma} a^\dagger_{i, \sigma} a_{i, \sigma}
     - h \sum_i (a^\dagger_{i, \uparrow} a_{i, \uparrow} -
               a^\dagger_{i, \downarrow} a_{i, \downarrow})
\end{align}
$$

$$
    U \sum_{k=1}^{N-1} a_k^\dagger a_k a_{k+1}^\dagger a_{k+1}
$$


If True, the repulsion term is replaced by:


$$
    U \sum_{k=1}^{N-1} (a_k^\dagger a_k - \frac12)
                       (a_{k+1}^\dagger a_{k+1} - \frac12)
$$


which is unchanged under a particle-hole transformation.
Default is False


## HubbardArgs

In [508]:
class HubbardArgs():
    def __init__(self, 
                x_dimension,
                y_dimension,
                tunneling,
                coulomb,
                chemical_potential=0.0,
                magnetic_field=0.0,
                periodic=True,
                spinless=False,
                particle_hole_symmetry=False,
                sc_gap=1.0,
                qsim_option=None):
        
        self.x_dimension = x_dimension
        self.y_dimension = y_dimension
        n_sites = x_dimension * y_dimension
        self.n_sites = n_sites
        self.n_qubits = 2*n_sites
        self.tunneling = tunneling
        self.coulomb = coulomb
        
        self.chemical_potential = chemical_potential
        self.magnetic_field = magnetic_field
        self.periodic = periodic
        self.spinless = spinless
        self.particle_hole_symmetry = particle_hole_symmetry
        self.sc_gap = sc_gap

        if not (qsim_option):
            self.qsim_option = qsim_option
        else:
            self.qsim_option = {'t': 4, 'f': 1}

        self.hopping_matrix = -tunneling * np.array([ \
                [ 0., 1., 1., 0.], \
                [ 1., 0., 0., 1.], \
                [ 1., 0., 0., 1.], \
                [ 0., 1., 1., 0.], \
            ])


[Quantum simulation of electronic structure  |  OpenFermion  |  Google Quantum AI](https://quantumai.google/openfermion/tutorials/intro_workshop_exercises)

In [509]:
# circuit hubbard
x_dimension=2
y_dimension=2
tunneling=1
coulomb=4
chemical_potential=0
magnetic_field=0
periodic=True
spinless=False
particle_hole_symmetry=False
qsim_option={'t':4}
sc_gap = 1.0
n_qubits = x_dimension*y_dimension*2


## bcs state, exponentiate

In [510]:
# bcs state circuit
mean_field_model = of.mean_field_dwave(
    x_dimension, y_dimension, tunneling, sc_gap, periodic)
quad_ham = of.get_quadratic_hamiltonian(mean_field_model)

qubits = cirq.LineQubit.range(n_qubits)
circuit = cirq.Circuit(of.circuits.prepare_gaussian_state(qubits, quad_ham))


In [511]:
# bcs e^(-iHt) circuit
function_args = HubbardArgs( 
                x_dimension,
                y_dimension,
                tunneling,
                coulomb,
                chemical_potential,
                magnetic_field,
                periodic,
                spinless,
                particle_hole_symmetry,
                qsim_option)

mean_field_model = of.mean_field_dwave(
    x_dimension, y_dimension, tunneling, sc_gap, periodic)
quad_ham = of.get_quadratic_hamiltonian(mean_field_model)


# Initialize qubits
qubits = cirq.LineQubit.range(n_qubits)

def _exponentiate_quad_ham(qubits, quad_ham, time):
    _, basis_change_matrix, _ = quad_ham.diagonalizing_bogoliubov_transform()
    orbital_energies, _ = quad_ham.orbital_energies()

    yield cirq.inverse(of.bogoliubov_transform(qubits, basis_change_matrix))
    for i in range(len(qubits)):
        yield cirq.rz(rads=-orbital_energies[i]*time).on(qubits[i])
    yield of.bogoliubov_transform(qubits, basis_change_matrix)

time = 0.5

circuit = cirq.Circuit(_exponentiate_quad_ham(qubits, quad_ham, time))


## fermion terms on hubbard

In [512]:
def _get_one_body_term_on_hubbard(hopping_matrix, n_sites):

    # get fermionOperator terms list in Hubbard model
    one_body_terms = []
    # spin up
    for i in range(n_sites):
        for j in range(i):
            hopping = hopping_matrix[i][j]
            # hopping = tunneling
            site_i = i*2
            site_j = j*2
            term_conjugated = of.FermionOperator(((site_i, 1), (site_j, 0)), coefficient=hopping) \
                + of.FermionOperator(((site_j, 1), (site_i, 0)), coefficient=hopping)
            one_body_terms.append(term_conjugated)
    # spin down
    for i in range(n_sites):
        for j in range(i):
            hopping = hopping_matrix[i][j]
            # hopping = tunneling
            site_i = i*2 + 1
            site_j = j*2 + 1
            term_conjugated = of.FermionOperator(((site_i, 1), (site_j, 0)), coefficient=hopping) \
                + of.FermionOperator(((site_j, 1), (site_i, 0)), coefficient=hopping)
            one_body_terms.append(term_conjugated)
            
    return one_body_terms

def _get_two_body_term_on_hubbard(coulomb, n_sites, n_qubits):
    # charge_matrix = coulomb * (np.diag([1] * n_sites)) # diagonal matrix
    two_body_terms = [
            of.FermionOperator(((i, 1), (i, 0), (i + 1, 1), (i + 1, 0)), coefficient=coulomb)
            for i in range(0, n_qubits, 2)
        ]
    return two_body_terms
    

## hubbard hamiltonian

In [513]:
# hubbard e^(-iHt) circuit
function_args = HubbardArgs( 
                x_dimension=2,
                y_dimension=2,
                tunneling=1.0,
                coulomb=4.0,
                chemical_potential=0.0,
                magnetic_field=0.0,
                periodic=True,
                spinless=False,
                particle_hole_symmetry=False,
                qsim_option={'t':4,'f':1})

fermion_hamiltonian = of.fermi_hubbard(
        function_args.x_dimension,
        function_args.y_dimension,
        function_args.tunneling,
        function_args.coulomb,
        function_args.chemical_potential,
        function_args.magnetic_field,
        function_args.periodic,
        function_args.spinless,
        function_args.particle_hole_symmetry
    )

n_sites = function_args.x_dimension * function_args.y_dimension
n_qubits = n_sites*2

# hopping_matrix = -tunneling * np.array([ \
#     [ 0., 1., 0., 0., 1., 0., 0., 0.], \
#     [ 1., 0., 1., 0., 0., 1., 0., 0.], \
#     [ 0., 1., 0., 1., 0., 0., 1., 0.], \
#     [ 0., 0., 1., 0., 0., 0., 0., 1.], \
#     [ 1., 0., 0., 0., 0., 1., 0., 0.], \
#     [ 0., 1., 0., 0., 1., 0., 1., 0.], \
#     [ 0., 0., 1., 0., 0., 1., 0., 1.], \
#     [ 0., 0., 0., 1., 0., 0., 1., 0.]  \
# ])
hopping_matrix = -tunneling * np.array([ \
    [ 0., 1., 1., 0.], \
    [ 1., 0., 0., 1.], \
    [ 1., 0., 0., 1.], \
    [ 0., 1., 1., 0.], \
])
fermion_hamiltonian_2 = sum(_get_one_body_term_on_hubbard(hopping_matrix, n_sites)) + sum(_get_two_body_term_on_hubbard(coulomb, n_sites, n_qubits))

print(fermion_hamiltonian)
print()
print(fermion_hamiltonian_2)
assert fermion_hamiltonian==fermion_hamiltonian_2


qubits = cirq.LineQubit.range(n_qubits)
time = 0.5
circuit = cirq.Circuit()
# circuit tunneling
for term in _get_one_body_term_on_hubbard(hopping_matrix, n_sites):
    term_qubit = of.qubit_operator_to_pauli_sum(of.jordan_wigner(term))
    circuit.append(cirq.PauliSumExponential(
        term_qubit,
        exponent= time,
        atol = 1e-08
    ))

# circuit coulomb
for term in _get_two_body_term_on_hubbard(coulomb, n_sites, n_qubits):
    term_qubit = of.qubit_operator_to_pauli_sum(of.jordan_wigner(term))
    circuit.append(cirq.PauliSumExponential(
        term_qubit,
        exponent= time,
        atol = 1e-08
    ))
# print(circuit.to_text_diagram())

4.0 [0^ 0 1^ 1] +
-1.0 [0^ 2] +
-1.0 [0^ 4] +
-1.0 [1^ 3] +
-1.0 [1^ 5] +
-1.0 [2^ 0] +
4.0 [2^ 2 3^ 3] +
-1.0 [2^ 6] +
-1.0 [3^ 1] +
-1.0 [3^ 7] +
-1.0 [4^ 0] +
4.0 [4^ 4 5^ 5] +
-1.0 [4^ 6] +
-1.0 [5^ 1] +
-1.0 [5^ 7] +
-1.0 [6^ 2] +
-1.0 [6^ 4] +
4.0 [6^ 6 7^ 7] +
-1.0 [7^ 3] +
-1.0 [7^ 5]

4.0 [0^ 0 1^ 1] +
-1.0 [0^ 2] +
-1.0 [0^ 4] +
-1.0 [1^ 3] +
-1.0 [1^ 5] +
-1.0 [2^ 0] +
4.0 [2^ 2 3^ 3] +
-1.0 [2^ 6] +
-1.0 [3^ 1] +
-1.0 [3^ 7] +
-1.0 [4^ 0] +
4.0 [4^ 4 5^ 5] +
-1.0 [4^ 6] +
-1.0 [5^ 1] +
-1.0 [5^ 7] +
-1.0 [6^ 2] +
-1.0 [6^ 4] +
4.0 [6^ 6 7^ 7] +
-1.0 [7^ 3] +
-1.0 [7^ 5]


## anzats

In [514]:
# anzats hubbard

class AnzatsBCSHubbard():
    def __init__(self, function_args, gamma, beta):
        # initialize circuit
        x = function_args.x_dimension
        y = function_args.y_dimension
        n_sites = function_args.n_sites
        tunneling=function_args.tunneling
        hopping_matrix=function_args.hopping_matrix
        coulomb=function_args.coulomb
        periodic=function_args.periodic
        sc_gap=function_args.sc_gap
        qubits = cirq.LineQubit.range(function_args.n_qubits)
        
        # |\psi 1> circuit
        mean_field_model = of.mean_field_dwave(
            x_dimension=x, y_dimension=y, tunneling=tunneling, sc_gap=sc_gap, periodic=periodic)
        quad_ham = of.get_quadratic_hamiltonian(mean_field_model)
        
        circuit = cirq.Circuit(
            of.circuits.prepare_gaussian_state(qubits, quad_ham))

        for index in range(gamma.size):
            # H2 (gamma) hubbard
            # circuit tunneling
            for term in _get_one_body_term_on_hubbard(hopping_matrix, n_sites):
                term_qubit = of.qubit_operator_to_pauli_sum(of.jordan_wigner(term))
                circuit.append(cirq.PauliSumExponential(
                    term_qubit,
                    exponent=gamma[index],
                    atol = 1e-08
                ))

            # circuit coulomb
            # term = of.jordan_wigner(sum(_get_two_body_term_on_hubbard(coulomb, n_sites, n_qubits)))
            term = sum(_get_two_body_term_on_hubbard(coulomb, n_sites, n_qubits))
            term_qubit = of.qubit_operator_to_pauli_sum(of.jordan_wigner(term))
            circuit.append(cirq.PauliSumExponential(
                term_qubit,
                exponent=gamma[index],
                atol = 1e-08
            ))
            # for term in _get_two_body_term_on_hubbard(coulomb, n_sites, n_qubits):
            #     term_qubit = of.qubit_operator_to_pauli_sum(of.jordan_wigner(term))
            #     circuit.append(cirq.PauliSumExponential(
            #         term_qubit,
            #         exponent=beta[index],
            #         atol = 1e-08
            #     ))
            # H1 (beta) bcs 
            circuit.append(
                cirq.Circuit(
                    _exponentiate_quad_ham(qubits, quad_ham, beta[index]
            )))

        self.circuit = circuit
        self.qubits = qubits
        self.gamma = gamma
        self.beta = beta



## expectation

In [515]:
# expectation hubbard 1
def get_expectation_bcs_hubbard(function_args, gamma, beta):
    # Initialize the anzats circuit for the Hubbard model
    anzats = AnzatsBCSHubbard(function_args=function_args, gamma=gamma, beta=beta)
    circuit = anzats.circuit
    simulator = qsimcirq.QSimSimulator(function_args.qsim_option)
    vector = simulator.simulate(circuit, qubit_order=anzats.qubits).state_vector()

    value = 0 + 0j

    # Calculate contribution from one-body terms (hopping terms)
    for term in _get_one_body_term_on_hubbard(function_args.hopping_matrix, function_args.n_sites):
        term_qubit = of.qubit_operator_to_pauli_sum(of.jordan_wigner(term))
        circuit_tunneling = anzats.circuit.copy()
        circuit_tunneling.append(term_qubit)  # Apply the hopping term to the copied circuit
        vector_tunneling = simulator.simulate(circuit_tunneling, qubit_order=anzats.qubits).state_vector()
        value_one = np.dot(vector_tunneling.conj(), vector)
        value += complex(value_one)

    # Calculate contribution from two-body terms (Coulomb interaction)
    term = sum(_get_two_body_term_on_hubbard(function_args.coulomb, function_args.n_sites, function_args.n_qubits))
    term_qubit = of.qubit_operator_to_pauli_sum(of.jordan_wigner(term))
    circuit_coulomb = anzats.circuit.copy()
    circuit_coulomb.append(term_qubit)  # Apply the Coulomb interaction to the copied circuit
    
    vector_coulomb = simulator.simulate(circuit_coulomb, qubit_order=anzats.qubits).state_vector()
    value_two = np.dot(vector_coulomb.conj(), vector)
    value += complex(value_two)

    return value


In [516]:
# expectation hubbard 2
def get_expectation_bcs_hubbard(function_args, gamma, beta):
    # Initialize the anzats circuit for the Hubbard model
    anzats = AnzatsBCSHubbard(function_args=function_args, gamma=gamma, beta=beta)
    circuit = anzats.circuit
    simulator = qsimcirq.QSimSimulator(function_args.qsim_option)
    vector = simulator.simulate(circuit, qubit_order=anzats.qubits).state_vector()

    value = 0 + 0j

    # # Calculate contribution from one-body terms (hopping terms)
    term = sum(_get_one_body_term_on_hubbard(function_args.hopping_matrix, function_args.n_sites))
    term_qubit = of.qubit_operator_to_pauli_sum(of.jordan_wigner(term))
    value_one = simulator.simulate_expectation_values(circuit, observables=[term_qubit])
    value += sum(value_one)

    # Calculate contribution from two-body terms (Coulomb interaction)
    term = sum(_get_two_body_term_on_hubbard(function_args.coulomb, function_args.n_sites, function_args.n_qubits))
    term_qubit = of.qubit_operator_to_pauli_sum(of.jordan_wigner(term))
    value_two = simulator.simulate_expectation_values(circuit, observables=[term_qubit])
    value += sum(value_two)

    return value


In [517]:
circuit = cirq.Circuit()
for term in term_qubit:
    print(term)
    print(type(term))
    circuit.append(term)
    
simulator = qsimcirq.QSimSimulator(function_args.qsim_option)
simulator.simulate(circuit)

I
<class 'cirq.ops.pauli_string.PauliString'>
-Z(q(7))
<class 'cirq.ops.pauli_string.PauliString'>
-Z(q(6))
<class 'cirq.ops.pauli_string.PauliString'>
Z(q(6))*Z(q(7))
<class 'cirq.ops.pauli_string.PauliString'>


measurements: (no measurements)

qubits: (cirq.LineQubit(6), cirq.LineQubit(7))
output vector: |00⟩

In [518]:
function_args = HubbardArgs( 
                x_dimension=x_dimension,
                y_dimension=y_dimension,
                tunneling=tunneling,
                coulomb=coulomb,
                chemical_potential=chemical_potential,
                magnetic_field=magnetic_field,
                periodic=periodic,
                spinless=spinless,
                particle_hole_symmetry=particle_hole_symmetry,
                sc_gap=sc_gap,
                qsim_option=qsim_option)

gamma = np.array([0.5, 0.5])
beta = np.array([0.5, 0.5])
anzats = AnzatsBCSHubbard(function_args=function_args, gamma=gamma, beta=beta)
circuit = anzats.circuit
# print(circuit)

simulator = qsimcirq.QSimSimulator(function_args.qsim_option)
result = simulator.simulate(circuit, qubit_order=anzats.qubits)
vector = simulator.simulate(circuit, qubit_order=anzats.qubits).state_vector()
print(result)

norm = np.dot(vector.conjugate(), vector)
print("norm: {}".format(norm))


measurements: (no measurements)

qubits: (cirq.LineQubit(0), cirq.LineQubit(1), cirq.LineQubit(2), cirq.LineQubit(3), cirq.LineQubit(4), cirq.LineQubit(5), cirq.LineQubit(6), cirq.LineQubit(7))
output vector: [ 3.06090191e-02-2.22427491e-02j  8.07059237e-08+6.79254200e-08j
 -2.64131277e-08+8.88573553e-08j  1.39724776e-01+4.32406627e-02j
 -4.88582081e-08+4.51222135e-08j  1.33685667e-08-5.08436093e-09j
  4.18366976e-02-6.46830723e-02j -3.02465004e-08-2.08773510e-09j
  1.86318405e-09+3.22158322e-09j -4.18366864e-02+6.46831915e-02j
 -3.25948299e-08-5.19646406e-08j  2.10187512e-08+1.37532838e-08j
  1.39724821e-01+4.32406366e-02j -9.84460780e-09-1.04922453e-08j
 -1.46046364e-09-3.53882079e-08j -8.43961313e-02-9.51967984e-02j
 -5.08650437e-08+8.73695960e-09j -3.52845291e-08-1.28370115e-09j
  9.96116176e-02+3.59908305e-02j  1.38254244e-08+5.58895970e-08j
 -3.03222336e-08-5.80043000e-08j -1.21761969e-08-5.33621103e-09j
  9.67415161e-08+2.12999574e-08j  2.35610043e-09-2.17107452e-08j
 -1.5305000

## optimizer

In [519]:
def partial_derivative_gamma(f, gamma, beta, i, h=1e-5, variable='gamma'):
    var_plus = np.array(gamma, dtype=float)
    var_minus = np.array(var_plus, dtype=float)
    var_plus[i] += h
    var_minus[i] -= h
    derivative = (f(gamma=var_plus, beta=beta).real - f(gamma=var_minus, beta=beta).real) / (2 * h)
    return derivative

def partial_derivative_beta(f, gamma, beta, i, h=1e-5, variable='gamma'):
    var_plus = np.array(beta, dtype=float)
    var_minus = np.array(var_plus, dtype=float)
    var_plus[i] += h
    var_minus[i] -= h
    derivative = (f(gamma=gamma, beta=var_plus).real - f(gamma=gamma, beta=var_minus).real) / (2 * h)
    return derivative

def gradient_parallel(pool, f, gamma, beta, h=1e-5):
    if not (gamma.size == beta.size):
        return None  # ベクトルサイズが一致しなければNoneを返す

    n = gamma.size
    args_gamma = [(f, gamma, beta, i, h, 'gamma') for i in range(n)]
    args_beta = [(f, gamma, beta, i, h, 'beta') for i in range(n)]
    grad_gamma = pool.starmap(partial_derivative_gamma, args_gamma)
    grad_beta = pool.starmap(partial_derivative_beta, args_beta)
    # pool.join()
    return np.array(grad_gamma), np.array(grad_beta)


def optimize_by_gradient_descent_multiprocess(function, initial_gamma, initial_beta, alpha, delta_gamma, delta_beta, iteration, figure=True, filepath="", pool=mp.Pool(2)):
    gamma, beta = initial_gamma.copy(), initial_beta.copy()

# with open(filepath, mode='a', newline='') as f:
    # writer = csv.writer(f)
    headline = ["iter", "energy"]
    for p in range(int(len(initial_gamma))):
        headline.append("gamma[{}]".format(p))
        headline.append("beta[{}]".format(p))
    print(headline)
    # writer.writerow(headline)

    for iter in range(iteration):
        grad_gamma, grad_beta = gradient_parallel(pool, function, gamma, beta, delta_gamma)
        # print(grad_beta, grad_gamma)
        gamma -= alpha * grad_gamma
        beta -= alpha * grad_beta
        energy = function(gamma=gamma, beta=beta)

        record = [iter, energy] + [val for pair in zip(gamma, beta) for val in pair]
        # writer.writerow(record)
        if figure:
            print(record)

    return gamma, beta




## case: x=2, y=2, Ut=4

In [520]:
function_args = HubbardArgs( 
                x_dimension=2,
                y_dimension=2,
                tunneling=1.0,
                coulomb=4.0,
                chemical_potential=0.0,
                magnetic_field=0.0,
                periodic=True,
                spinless=False,
                particle_hole_symmetry=False,
                qsim_option={'t':4,'f':1})

gamma = np.array([0.5, 0.5, 0.5, 0.5])
beta = np.array([0.5, 0.5, 0.5, 0.5])
anzats = AnzatsBCSHubbard(function_args=function_args, gamma=gamma, beta=beta)
circuit = anzats.circuit
# print(circuit)

simulator = qsimcirq.QSimSimulator(function_args.qsim_option)
# result = simulator.simulate(circuit, qubit_order=anzats.qubits)
vector = simulator.simulate(circuit, qubit_order=anzats.qubits).state_vector()
# print(result)

norm = np.dot(vector.conjugate(), vector)
print("norm: {}".format(norm))


# value = get_expectation_bcs_hubbard(function_args, gamma, beta)
f = partial(get_expectation_bcs_hubbard, function_args=function_args)
value = f(gamma=gamma, beta=beta)
print("expectation on bcs_hubbard: {}".format(value))


norm: (0.9999666810035706+0j)
expectation on bcs_hubbard: (4.555332004849164+1.2331966781075191e-08j)


## case: x=2, y=2, Ut=4 optimize

In [521]:
function_args = HubbardArgs( 
                x_dimension=1,
                y_dimension=4,
                tunneling=1.0,
                coulomb=4.0,
                chemical_potential=0.0,
                magnetic_field=0.0,
                periodic=True,
                spinless=False,
                particle_hole_symmetry=False,
                sc_gap=1.0,
                qsim_option={'t':4,'f':1})

p = 2
initial_gamma = np.array([0.5 for i in range(p)])
initial_beta = np.array([0.5 for i in range(p)])
alpha = 0.01
delta_gamma = 0.001
delta_beta  = 0.001
iteration = 10
pool = mp.Pool(4)

f = partial(get_expectation_bcs_hubbard, function_args=function_args)
gamma, beta = optimize_by_gradient_descent_multiprocess(
    function=f, 
    initial_gamma=initial_gamma,
    initial_beta=initial_beta, 
    alpha=alpha,
    delta_gamma=delta_gamma,
    delta_beta=delta_beta,
    iteration=iteration,
    figure=False,
    filepath="",
    pool=pool)

pool.close()
pool.join()
print(gamma, beta)


['iter', 'energy', 'gamma[0]', 'beta[0]', 'gamma[1]', 'beta[1]']
[0.28168779 0.06413397] [0.52583693 0.64942334]


In [522]:
def exact_eigenvalue( 
                x_dimension=1,
                y_dimension=4,
                tunneling=1,
                coulomb=4,
                chemical_potential=0.0,
                magnetic_field=0.0,
                periodic=True,
                spinless=False,
                particle_hole_symmetry=False,
                qsim_option=None, 
                particle_n=4):
    function_args = HubbardArgs( 
                    x_dimension,
                    y_dimension,
                    tunneling,
                    coulomb,
                    chemical_potential,
                    magnetic_field,
                    periodic,
                    spinless,
                    particle_hole_symmetry,
                    qsim_option)

    # define a Hubbard hamiltonian and convert into qubit operator through jordan-wigner transformation
    fermion_hamiltonian = of.fermi_hubbard(
        function_args.x_dimension,
        function_args.y_dimension,
        function_args.tunneling,
        function_args.coulomb,
        function_args.chemical_potential,
        function_args.magnetic_field,
        function_args.periodic,
        function_args.spinless,
        function_args.particle_hole_symmetry
    )
    qubit_hamiltonian = of.transforms.jordan_wigner(fermion_hamiltonian)

    # Convert to Scipy sparse matrix
    hamiltonian_jw_sparse = of.get_sparse_operator(qubit_hamiltonian)

    # Compute ground energy
    eigs, _ = linalg.eigsh(hamiltonian_jw_sparse, k=1, which="SA")
    ground_energy = eigs[0]
    # print("Ground_energy: {}".format(ground_energy))
    # print("JWT transformed Hamiltonian:")
    # print(qubit_hamiltonian)

    energy_on_n = of.linalg.jw_get_ground_state_at_particle_number(
        hamiltonian_jw_sparse, particle_n
    )
    
    # print(energy_on_n[0])
    return ground_energy, energy_on_n[0]



check if energy values equal to its exact one

Ut=2and4, on half-filling
- [x] W=1 L=2 
- [x] W=1 L=4 
- [x] W=1 L=8
- [x] W=2 L=2
- [ ] W=2 L=4


In [523]:
print("x y n c energy, energy_on_n")
for x in [1]:
    for y in [2]:
        for n in [1,2]:
            for coulomb in [2,4]:
                print(x, y, n, coulomb, exact_eigenvalue(
                    x_dimension=x, y_dimension=y,coulomb=coulomb, particle_n=n
                ))

x y n c energy, energy_on_n
1 2 1 2 (-1.2360679774997916, -1.0000000000000004)
1 2 1 4 (-1.0000000000000007, -1.0)
1 2 2 2 (-1.236067977499792, -1.236067977499789)
1 2 2 4 (-1.0000000000000002, -0.8284271247461908)


In [524]:
print("x y n c energy, energy_on_n")
for x in [1]:
    for y in [4]:
        for n in [2,4,8]:
            for coulomb in [2,4]:
                print(x, y, n, coulomb, exact_eigenvalue(
                    x_dimension=x, y_dimension=y,coulomb=coulomb, particle_n=n
                ))

x y n c energy, energy_on_n
1 4 2 2 (-3.6272130052966562, -3.6272130052966576)
1 4 2 4 (-3.418550718873851, -3.4185507188738424)
1 4 4 2 (-3.627213005296671, -2.828427124746194)
1 4 4 4 (-3.4185507188738504, -2.1027484834620767)
1 4 8 2 (-3.6272130052966585, 8.0)
1 4 8 4 (-3.41855071887383, 16.0)


In [525]:
print("x y n c energy, energy_on_n")
for x in [1]:
    for y in [8]:
        for n in [2,4,8]:
            for coulomb in [2,4]:
                print(x, y, n, coulomb, exact_eigenvalue(
                    x_dimension=x, y_dimension=y,coulomb=coulomb, particle_n=n
                ))

x y n c energy, energy_on_n


1 8 2 2 (-7.826311280176374, -3.8549403676467593)
1 8 2 4 (-6.672195997058443, -3.800539311244656)
1 8 4 2 (-7.826311280176394, -6.261566147504009)
1 8 4 4 (-6.672195997058456, -5.9517026573552805)
1 8 8 2 (-7.826311280176432, -6.568192162874722)
1 8 8 4 (-6.672195997058463, -4.603526299989214)


In [526]:
print("x y n c energy, energy_on_n")
for x in [2]:
    for y in [2,4]:
        for n in [4,8]:
            for coulomb in [2,4]:
                print(x, y, n, coulomb, exact_eigenvalue(
                    x_dimension=x, y_dimension=y,coulomb=coulomb, particle_n=n
                ))

x y n c energy, energy_on_n
2 2 4 2 (-3.627213005296659, -2.8284271247461876)
2 2 4 4 (-3.418550718873858, -2.1027484834620758)
2 2 8 2 (-3.6272130052966673, 8.0)
2 2 8 4 (-3.4185507188738415, 16.0)
2 4 4 2 (-8.478303296869603, -7.431230836069726)
2 4 4 4 (-7.612474927765829, -7.092266429238447)
2 4 8 2 (-8.478303296869544, -8.47830329686961)
2 4 8 4 (-7.612474927765857, -5.954236681057321)


In [527]:
print("x y n c energy, energy_on_n")
for x in [2]:
    for y in [2,4]:
        for n in [4,8]:
            for coulomb in [2,4]:
                print(x, y, n, coulomb, exact_eigenvalue(
                    x_dimension=x, y_dimension=y,coulomb=coulomb, particle_n=n,
                    periodic=False
                ))

x y n c energy, energy_on_n
2 2 4 2 (-3.627213005296666, -2.8284271247461965)
2 2 4 4 (-3.418550718873855, -2.102748483462083)
2 2 8 2 (-3.6272130052966607, 8.0)
2 2 8 4 (-3.4185507188738486, 16.0)
2 4 4 2 (-7.912602325322796, -7.713287732110458)
2 4 4 4 (-7.274487013496225, -7.274487013496268)
2 4 8 2 (-7.9126023253228, -7.11744658031349)
2 4 8 4 (-7.274487013496248, -5.0125031526570485)
